
# Getting Started with the AMC Quickstart

#### Import Libraries

In [ ]:
import boto3
import json
import pandas as pd
from client_manager_microservices.tps.atsclientslibraries.clientmgr import customer
from datalake_hydration_microservices.wfm.atsclientslibraries.workflows import workflows
from datalake_hydration_microservices.wfm.atsclientslibraries.workflow_invoke import workflowInvoke

#### Define Global Configurations

In [ ]:
ENV = "dev" # Change to your Default Environment

## The Team Name configured in the SDLF Data Lake platform
# This is the same value passed in the 'data_lake_parameters' team of ddk.json file
ATS_TEAM_NAME = "<INSERT TEAM NAME>" 

## The Dataset Name configured in the SDLF Data Lake platform
# This is the same value passed in the 'data_lake_parameters' dataset of ddk.json file
amc_dataset_name = "<INSERT DATASET NAME>" 

aws_region = 'us-east-1' # Change to your Default Region

customerId = '<INSERT CUSTOMER ID>'

# The AMC Instance Information Can Be Found on Your AMC UI Page
amc_api_endpoint = "<ENTER AMC API ENDPOINT URL>"

amc_s3_bucket_name = "<ENTER AMC S3 BUCKET NAME>"

amc_data_upload_acct = "<ENTER ACCOUNT ID>"

## OPTIONAL: A Comma seperated list of Advertiser IDs. This can be retrieved from the ticket raised to the AMC team. 
## It is the list provided while raising the AMC instance

advertiser_ids = ""

## OPTIONAL: A Comma seperated list of Entity IDs. This can be retrieved from the ticket raised to the AMC team. 
## It is the list provided while raising the AMC instance
entity_ids = ""


if aws_region == '':
    aws_region = str(boto3.Session().region_name)
print("Region : " + aws_region)

#### Step 1: Onboard A New Client

Tenant Provisioning Service (TPS) is used for onboarding clients for each team space. Each Client should be configured in the same AWS Region as the AMC Instance. Each client is defined by:

1. An AMC Instance
2. Corresponding profiles ids which are grouped for this customer according the advertiserids used for setting up the AMC instance

The following notebook cells will define your client configuration, onboard your client, and create the AMC S3 Bucket for that client

##### Define your Client Configuration

Run the below cell to setup and verify the client configuration. 

Refer to the `client_manager_microservices/client_manager_adminstrator_sample` notebook for more information on the configuration parameters.

In [ ]:
customer_details = {
    "customer_id":customerId,
    "customer_name":"DemoCustomer",
    "customer_type": "ENDEMIC",
    "region":aws_region,
    "amc":{
        "amc_dataset_name":amc_dataset_name,
        "endpoint_url": amc_api_endpoint,
        "aws_orange_account_id": amc_data_upload_acct,
        "bucket_name": amc_s3_bucket_name,
        "advertiser_ids": advertiser_ids,
        "entity_ids": entity_ids
    }
}

print(json.dumps(customer_details, indent=4))

##### Submit Your Client Configuration to Create The AMC S3 Bucket

Running the cells below will start the process of onboarding your client and creating the AMC S3 Bucket for your client.

In [ ]:
dynamodb_resp_wr = customer.set_customers_config(customer_details=customer_details, ATS_TEAM_NAME=ATS_TEAM_NAME, ENV=ENV)
dynamodb_resp_wr

_Wait a few minutes for the AMC S3 Bucket to be delpoyed BEFORE moving to Step #2._

_You Can Verify the Status by going to AWS CloudFormation and waiting until the stack name amc-&lt;DatasetName&gt;-instance-&lt;customerId&gt;" Status is marked Create Complete._



#### Step 2: Define An AMC Workflow Query and Set An AMC Workflow Record. 

Workflow Manager (WFM) Service is used to manage and schedule AMC workflows. The following notebook cells with walkthrough the process of creating an AMC workflow query and creating an AMC Workflow record for the specified query.

The query used here is the DSP Display, Streaming TV and Sponsored Products Three Way Overlap Query from the Interactive Query Library (IQL) in the AMC UI.

This IQ will allow you to analyze reach and performance across up to three different custom defined groupings, measuring the impact of a full-funnel strategy across multiple ad tactics, including DSP Display, Streaming TV and Sponsored Products.


In [ ]:
# -- Instructional Query: DSP Display, Streaming TV and Sponsored Products Three Way Overlap

# -- Exposure Group Selection: P1-Display, P2-Streaming TV, P3-Sponsored Products

# --------------------------------------------------------------
# -- Table for ad attributed purchases
# -- Change display campaign (campaign_id) and sponsored products campaign (campaign name) here
# --------------------------------------------------------------
amc_query = """

with p1_ids as 
(SELECT campaign_id FROM
(VALUES
/*UPDATE DSP Display campaign_ids here*/ 
(111122222)) as b(campaign_id) 
),

p2_ids as 
(SELECT campaign_id FROM
(VALUES
/*UPDATE Streaming TV campaign_ids here*/ 
(111122222),
(111122222)) as b(campaign_id) 
),

p3_ids as 
(SELECT campaign FROM
(VALUES
/*UPDATE Sponsored Ads campaign names here. The name must be used. We do not have a customer-facing campaign_id available yet for Sponsored Ads*/ 
('SP_campaign_name1'),
('SP_campaign_name2'),
('SP_campaign_name3'),
('SP_campaign_name4')) as b(campaign)
),

purchases_temp as (
SELECT
  user_id
  ,campaign_id
  ,campaign
  ,conversion_event_dt
  ,sum(purchases) as purchases
  ,sum(total_purchases_clicks) as total_purchases_clicks
  ,sum(product_sales) as sales
FROM amazon_attributed_events_by_traffic_time a
WHERE (campaign_id IN (SELECT campaign_id from p1_ids) OR
       campaign_id IN (SELECT campaign_id from p2_ids) OR
       campaign IN (SELECT campaign from p3_ids))
GROUP BY 1,2,3,4
),

purchases as (

SELECT
  user_id
  ,conversion_event_dt
  , SUM(purchases) AS p1_purchases
  , 0 AS p2_purchases
  , 0 AS p3_purchases
  , SUM(sales) as sales
FROM purchases_temp a
WHERE campaign_id IN (SELECT campaign_id from p1_ids)
GROUP BY 1,2

UNION ALL

SELECT
  user_id
  ,conversion_event_dt
  , 0 AS p1_purchases
  , SUM(purchases) AS p2_purchases
  , 0 AS p3_purchases
  , SUM(sales) as sales
FROM purchases_temp a
WHERE campaign_id IN (SELECT campaign_id from p2_ids)
GROUP BY 1,2

UNION ALL

SELECT
  user_id
  ,conversion_event_dt
  , 0 AS p1_purchases
  , 0 AS p2_purchases
  , SUM(total_purchases_clicks) AS p3_purchases
  , SUM(sales) as sales
FROM purchases_temp a
JOIN (SELECT campaign FROM p3_ids) b on a.campaign = b.campaign
GROUP BY 1,2
),

purchases_last_total as
(
SELECT
user_id
, max(conversion_event_dt)                  AS conversion_event_dt_last
, sum(p1_purchases + p2_purchases + p3_purchases) AS purchases
, sum(p1_purchases)                         AS p1_purchases
, sum(p2_purchases)                         AS p2_purchases
, sum(p3_purchases)                         AS p3_purchases
, sum(sales)                                AS sales
from purchases
WHERE p1_purchases+p2_purchases+p3_purchases > 0
GROUP BY 1
),


impressions_temp as (
SELECT
user_id
, campaign_id
, campaign
, UUID()           AS impression_id
, impression_dt
, impressions AS impressions
FROM dsp_impressions a
WHERE (campaign_id IN (SELECT campaign_id from p1_ids) OR
       campaign_id IN (SELECT campaign_id from p2_ids))
AND impressions > 0

UNION all

SELECT 
user_id
, 0 as campaign_id
, a.campaign
, UUID()           AS impression_id
, event_dt         AS impression_dt
, impressions AS impressions
FROM sponsored_ads_traffic a
JOIN (SELECT campaign FROM p3_ids) b on a.campaign = b.campaign
WHERE impressions > 0
),

    impressions AS (

SELECT user_id
     , impression_id
     , impression_dt    
     , 'p1' AS IMP_ad_type
     , sum(impressions) AS impressions
FROM impressions_temp
WHERE campaign_id IN (SELECT campaign_id from p1_ids)
GROUP BY 1, 2, 3, 4

UNION ALL

SELECT user_id
     , impression_id
     , impression_dt
     , 'p2' AS IMP_ad_type
     , sum(impressions) AS impressions
FROM impressions_temp
WHERE campaign_id IN (SELECT campaign_id from p2_ids)
GROUP BY 1, 2, 3, 4

UNION ALL

SELECT user_id
     , impression_id
     , impression_dt
     , 'p3'             AS IMP_ad_type
     , sum(impressions) AS impressions
FROM impressions_temp a
JOIN (SELECT campaign FROM p3_ids) b on a.campaign = b.campaign
GROUP BY 1, 2, 3, 4),

impressions_first_total as
(
SELECT
user_id
,imp_ad_type
,min(impression_dt) as impression_dt_first
,sum(impressions) as impressions
from impressions
GROUP BY 1,2
),

combined as (
SELECT
  imp.user_id as imp_user
  ,pur.user_id as pur_user
  ,imp.imp_ad_type
  ,pur.purchases
  ,pur.p1_purchases
  ,pur.p2_purchases
  ,pur.p3_purchases
  ,imp.impressions
  ,pur.sales
    ,CASE WHEN conversion_event_dt_last IS NULL THEN 'no purchase'
    WHEN conversion_event_dt_last > impression_dt_first THEN 'pre-purchase'
    WHEN conversion_event_dt_last < impression_dt_first THEN 'post-purchase'
    ELSE 'other'
    END AS impression_timing
FROM purchases_last_total pur
FULL OUTER JOIN impressions_first_total imp ON pur.user_id = imp.user_id
),

binned_data as (
SELECT
 imp_user
 , pur_user
  ,SUM(IF(impression_timing = 'no purchase' AND imp_ad_type = 'p1', impressions, 0)) as no_purch_p1_impressions
  ,SUM(IF(impression_timing = 'pre-purchase' AND imp_ad_type = 'p1', impressions, 0)) as pre_purch_p1_impressions
  ,SUM(IF(impression_timing = 'post-purchase' AND imp_ad_type = 'p1', impressions, 0)) as post_purch_p1_impressions
  ,SUM(IF(impression_timing = 'other' AND imp_ad_type = 'p1', impressions, 0)) as other_p1_impressions
  ,SUM(IF(imp_ad_type = 'p1', impressions, 0)) as total_p1_impressions
  ,SUM(IF(impression_timing = 'no purchase' AND imp_ad_type = 'p2', impressions, 0)) as no_purch_p2_impressions
  ,SUM(IF(impression_timing = 'pre-purchase' AND imp_ad_type = 'p2', impressions, 0)) as pre_purch_p2_impressions
  ,SUM(IF(impression_timing = 'post-purchase' AND imp_ad_type = 'p2', impressions, 0)) as post_purch_p2_impressions
  ,SUM(IF(impression_timing = 'other' AND imp_ad_type = 'p2', impressions, 0)) as other_p2_impressions
  ,SUM(IF(imp_ad_type = 'p2', impressions, 0)) as total_p2_impressions
  ,SUM(IF(impression_timing = 'no purchase' AND imp_ad_type = 'p3', impressions, 0)) as no_purch_p3_impressions
  ,SUM(IF(impression_timing = 'pre-purchase' AND imp_ad_type = 'p3', impressions, 0)) as pre_purch_p3_impressions
  ,SUM(IF(impression_timing = 'post-purchase' AND imp_ad_type = 'p3', impressions, 0)) as post_purch_p3_impressions
  ,SUM(IF(impression_timing = 'other' AND imp_ad_type = 'p3', impressions, 0)) as other_p3_impressions
  ,SUM(IF(imp_ad_type = 'p3', impressions, 0)) as total_p3_impressions
  ,MAX(purchases) as purchases
  ,MAX(p1_purchases) as p1_purchases
  ,MAX(p2_purchases) as P2_purchases
  ,MAX(p3_purchases) as p3_purchases
  ,MAX(sales) as sales
FROM combined
GROUP BY 1,2
),

users_purchased as
(
SELECT
imp_user
, pur_user
, no_purch_p1_impressions
, pre_purch_p1_impressions
, post_purch_p1_impressions
, other_p1_impressions
, total_p1_impressions
, no_purch_p2_impressions
, pre_purch_p2_impressions
, post_purch_p2_impressions
, other_p2_impressions
, total_p2_impressions
, no_purch_p3_impressions
, pre_purch_p3_impressions
, post_purch_p3_impressions
, other_p3_impressions
, total_p3_impressions
, purchases
from binned_data
WHERE purchases > 0
),

users_purchased_exp as
(
SELECT
IF((pre_purch_p1_impressions > 0 AND pre_purch_p2_impressions > 0 AND pre_purch_p3_impressions > 0) OR  (no_purch_p1_impressions > 0 AND no_purch_p2_impressions > 0 AND no_purch_p3_impressions > 0 ), 'all',
IF((pre_purch_p1_impressions > 0 AND pre_purch_p2_impressions > 0 AND pre_purch_p3_impressions = 0) OR  (no_purch_p1_impressions > 0 AND no_purch_p2_impressions > 0 AND no_purch_p3_impressions = 0 ), 'p1&p2',
IF((pre_purch_p1_impressions > 0 AND pre_purch_p2_impressions = 0 AND pre_purch_p3_impressions > 0) OR  (no_purch_p1_impressions > 0 AND no_purch_p2_impressions = 0 AND no_purch_p3_impressions > 0 ), 'p1&p3',
IF((pre_purch_p1_impressions = 0 AND pre_purch_p2_impressions > 0 AND pre_purch_p3_impressions > 0) OR  (no_purch_p1_impressions = 0 AND no_purch_p2_impressions > 0 AND no_purch_p3_impressions > 0 ), 'p2&p3',
IF((pre_purch_p1_impressions > 0 OR no_purch_p1_impressions > 0) AND (pre_purch_p2_impressions = 0 AND no_purch_p2_impressions = 0) AND (pre_purch_p3_impressions = 0 AND no_purch_p3_impressions = 0), 'p1',
IF((pre_purch_p1_impressions = 0 AND no_purch_p1_impressions = 0) AND (pre_purch_p2_impressions > 0 OR no_purch_p2_impressions > 0) AND (pre_purch_p3_impressions = 0 AND no_purch_p3_impressions = 0), 'p2',
IF((pre_purch_p1_impressions = 0 AND no_purch_p1_impressions = 0) AND (pre_purch_p2_impressions = 0 AND no_purch_p2_impressions = 0) AND (pre_purch_p3_impressions > 0 OR no_purch_p3_impressions > 0), 'p3',
IF((pre_purch_p1_impressions = 0 AND no_purch_p1_impressions = 0) AND (pre_purch_p2_impressions = 0 AND no_purch_p2_impressions = 0) AND (pre_purch_p3_impressions = 0 AND no_purch_p3_impressions = 0) , 'none', 'NA'))))))))
AS exposure_group
,COUNT(distinct imp_user) as users_that_purchased_with_impressions
,COUNT(distinct pur_user) as users_that_purchased
FROM users_purchased
GROUP BY exposure_group
),

users_all_exp as
(SELECT
  IF((pre_purch_p1_impressions > 0 AND pre_purch_p2_impressions > 0 AND pre_purch_p3_impressions > 0) OR  (no_purch_p1_impressions > 0 AND no_purch_p2_impressions > 0 AND no_purch_p3_impressions > 0 ), 'all',
  IF((pre_purch_p1_impressions > 0 AND pre_purch_p2_impressions > 0 AND pre_purch_p3_impressions = 0) OR  (no_purch_p1_impressions > 0 AND no_purch_p2_impressions > 0 AND no_purch_p3_impressions = 0 ), 'p1&p2',
  IF((pre_purch_p1_impressions > 0 AND pre_purch_p2_impressions = 0 AND pre_purch_p3_impressions > 0) OR  (no_purch_p1_impressions > 0 AND no_purch_p2_impressions = 0 AND no_purch_p3_impressions > 0 ), 'p1&p3',
  IF((pre_purch_p1_impressions = 0 AND pre_purch_p2_impressions > 0 AND pre_purch_p3_impressions > 0) OR  (no_purch_p1_impressions = 0 AND no_purch_p2_impressions > 0 AND no_purch_p3_impressions > 0 ), 'p2&p3',
  IF((pre_purch_p1_impressions > 0 OR no_purch_p1_impressions > 0) AND (pre_purch_p2_impressions = 0 AND no_purch_p2_impressions = 0) AND (pre_purch_p3_impressions = 0 AND no_purch_p3_impressions = 0), 'p1',
  IF((pre_purch_p1_impressions = 0 AND no_purch_p1_impressions = 0) AND (pre_purch_p2_impressions > 0 OR no_purch_p2_impressions > 0) AND (pre_purch_p3_impressions = 0 AND no_purch_p3_impressions = 0), 'p2',
  IF((pre_purch_p1_impressions = 0 AND no_purch_p1_impressions = 0) AND (pre_purch_p2_impressions = 0 AND no_purch_p2_impressions = 0) AND (pre_purch_p3_impressions > 0 OR no_purch_p3_impressions > 0), 'p3',
  IF((pre_purch_p1_impressions = 0 AND no_purch_p1_impressions = 0) AND (pre_purch_p2_impressions = 0 AND no_purch_p2_impressions = 0) AND (pre_purch_p3_impressions = 0 AND no_purch_p3_impressions = 0) , 'none', 'NA'))))))))
        AS exposure_group
,COUNT(distinct imp_user) as impression_reach
,SUM(purchases) as purchases
,SUM(p1_purchases) as p1_purchases
,SUM(p2_purchases) as P2_purchases
,SUM(p3_purchases) as p3_purchases
,SUM(no_purch_p1_impressions) as no_purch_p1_impressions
,SUM(pre_purch_p1_impressions) as pre_purch_p1_impressions
,SUM(post_purch_p1_impressions) as post_purch_p1_impressions
,SUM(other_p1_impressions) as other_p1_impressions
,SUM(total_p1_impressions) as total_p1_impressions
,SUM(no_purch_p2_impressions) as no_purch_p2_impressions
,SUM(pre_purch_p2_impressions) as pre_purch_p2_impressions
,SUM(post_purch_p2_impressions) as post_purch_p2_impressions
,SUM(other_p2_impressions) as other_p2_impressions
,SUM(total_p2_impressions) as total_p2_impressions
,SUM(no_purch_p3_impressions) as no_purch_p3_impressions
,SUM(pre_purch_p3_impressions) as pre_purch_p3_impressions
,SUM(post_purch_p3_impressions) as post_purch_p3_impressions
,SUM(other_p3_impressions) as other_p3_impressions
,SUM(total_p3_impressions) as total_p3_impressions
,SUM(sales) as sales
FROM binned_data
GROUP BY exposure_group
)

SELECT
a.exposure_group
,users_that_purchased
,impression_reach
,purchases
,sales
FROM users_all_exp a
LEFT JOIN users_purchased_exp p
ON a.exposure_group = p.exposure_group
"""

print (amc_query)

__NOTE__: This is just one example of a workflow query you can run on your AMC Instance. Refer to the Interactive Query Library (IQL) in the AMC UI for a list of other queries for different use cases. Queries can also be customized for you unique use case as well.

##### Create The AMC Workflow Record

Run the below cell to setup and verify the workflow configuration (default configuration values are already populated). 

Refer to the `datalake_hydration_microservices/workflows_wfm_sample` notebook for more information on the workflow configuration parameters.



In [ ]:
# Workflow record
workflow = {
  "customerId": customerId,
  "defaultSchedule": {
    "automaticDeploySchedule": False,
    "Description": "Runs the display_streaming_sp workflow. ",
    "Input": {
      "payload": {
        "timeWindowEnd": "today(-1)",
        "timeWindowStart": "today(-8)",
        "timeWindowType": "EXPLICIT",
        "workflow_executed_date": "now()"
      }
    },
    "Name": "display_streaming_sp",
  },
  "filteredMetricsDiscriminatorColumn": "filtered",
  "sqlQuery": amc_query,
  "version": 1,
  "workflowId": "display_streaming_sp_v1",
  "workflowMetaData": {
    "automaticDeployWorkflow": True,
    "endemicType": "ENDEMIC"
  },
  "workflowType": "ENDEMIC|NON-ENDEMIC"
}

print(json.dumps(workflow, indent=4))

In [ ]:
dynamodb_resp_wr = workflows.set_workflow_record(workflow_details=workflow, ATS_TEAM_NAME=ATS_TEAM_NAME, ENV=ENV)
dynamodb_resp_wr

#### Step 3: Invoke the AMC Workflow to be Executed Ad Hoc

Run the below cell to set up your workflow exeuction configuration (default configuration values are already populated). 

Refer to the `datalake_hydration_microservices/workflows_invoke_wfm_sample` notebook for more information on the workflow exeuction configuration parameters.

In [ ]:
# Workflow record
workflow =  {
  "customerId": customerId,
  "Description": "Runs the display_streaming_sp workflow looking back to 7 days prior",
  "Input": {
    "payload": {
      "timeWindowEnd": "today(-1)",
      "timeWindowStart": "today(-8)",
      "timeWindowType": "EXPLICIT",
      "workflow_executed_date": "now()",
      "workflowId": "display_streaming_sp_v1"
    }
  },
  "Name": f'wfm-{customerId}-display_streaming_sp'
}

In [ ]:
response = workflowInvoke.invoke_workflow(workflow, ATS_TEAM_NAME, ENV)
response

__NOTE__: In this example we are invoking the display_streaming_sp workflow to be run once. Workflows can also be set up to run on pre-defined schedules custom to your use case. Refer to the `workflowLibrary_wfm_sample` and `workflowSchedules_wfm_sample` notebooks for more information.

#### Your workflow is now being processed and executed. 
Once the workflow has run, data will be uploaded to your AMC S3 Bucket with the results of the workflow query and processed through the data lake. 

- Continue exploring the notebooks in the `client_manager_microservices` folder for further documentation on how to onboard new clients

- Continue exploring the notebooks in the `datalake_hydration_microservices` folder for further documentation on how to schedule and manage your workflows.

### Optional : Data Visualization Using Workflow Query Results

Before you begin setting up data visualizations please wait until the workflow query has been executed and data has been populated in S3 and processed by the data lake. This can take around 30 minutes for data to be populated on S3 and processed through the data lake.

Once you have your query results, run the below cell to visualize the data you have gathered:

#### Data Visualization Using QuickSight Dashboards

If you require more advanced and customizable data visualizations consider using Amazon QuickSight as your BI Tool.  With Amazon QuickSight you can perform advanced analytics, gather machine learning (ML) insights and embed interactive visualizations and dashboards with natural language query capabilities. Refer to the AMC QuickStart FAQ to find more information on how to set up your own Amazon QuickSight Dashboards.